# MVDIA - Practical Assignment

**Authors**: Anton Helminen & Veikka Immonen

## Load modules

Modules used for the system. To install: `pip install [module]`

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import torch
from torch import nn
from torchvision.transforms import v2
from rtmlib import Wholebody, draw_skeleton

# Custom libraries
from Feature_extraction import opt_flow

## Device

Define preferred device for the classification

In [2]:
device = 'cuda' # or 'cpu'
torch.set_default_device(device)

## Load models and processing functionality

In [25]:
wholebody = Wholebody(
    to_openpose=False,
    mode='balanced',
    backend='onnxruntime', device=device
)

model = torch.load('model.pt').to(device)
model.eval()

transform = v2.Compose([
    v2.ToImage(),
    v2.Resize((128, 128)),
    v2.ToDtype(torch.float32, scale=True),
])

2024-03-29 16:20:58.231814983 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-03-29 16:20:58.231830243 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2024-03-29 16:20:58.505237524 [W:onnxruntime:, session_state.cc:1162 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-03-29 16:20:58.505254514 [W:onnxruntime:, session_state.cc:1164 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


load /home/veikka/.cache/rtmlib/hub/checkpoints/yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load /home/veikka/.cache/rtmlib/hub/checkpoints/rtmw-x_simcc-cocktail13_pt-ucoco_270e-256x192-fbef0d61_20230925.onnx with onnxruntime backend


## Classification function

```def classify(image_stream: list[ndarray]) -> int```

- `image_stream`: video sample as a list of images (`ndarray`, `dtype=uint8`)
- Returns: class from 1 to 32 as integer, -1 if fails.

Recommended: images are opened using `cv2.imread` without any color channel manipulation.

In [65]:
def classify(image_stream):

    try:
        pre_processed_images = []
        for image in image_stream:
            keypoints, scores = wholebody(image)
            # Skeleton image
            img_spooky = np.zeros(image.shape, dtype=np.uint8)
            img_spooky = draw_skeleton(img_spooky, keypoints, scores, kpt_thr=0.5)
            pre_processed_images.append(img_spooky)
        # Added optical flow
        if (len(pre_processed_images) > 1):
            image_final = opt_flow(pre_processed_images)
        else:
            image_final = pre_processed_images[0]
    except:
        print("Sample preprocessing failed, halt...")
        return -1

    image_final = cv2.cvtColor(image_final, cv2.COLOR_BGR2RGB)
    reshaped = transform(image_final).unsqueeze(0)
    with torch.no_grad():
        output = model(reshaped).squeeze()

    return output.argmax().cpu().item() + 1

## Test

Sample is from training set, class 20

In [67]:
sample_path = './test_sample/557'
sample = [cv2.imread(f'{sample_path}.{i}.jpg') for i in range(7)]
classify(sample)

20